In [10]:
import pandas as pd
import pandas_datareader.data as web
import datetime
import yfinance as yf

# Definir empresas
companies = ['AOS', 'AFL', 'ABT', 'PTC']

# Definir rango de fechas
start_date = datetime.datetime.now() - datetime.timedelta(days=33*365)
end_date = datetime.datetime.now()

# Crear un diccionario vacío para almacenar los dataframes de las empresas
dataframes = {}

# Obtener los datos históricos para cada empresa y agregarlos al diccionario
for company in companies:
    df = yf.download(company, start=start_date, end=end_date)
    df['Ticker'] = company
    dataframes[company] = df

# Crear las columnas "Rendimiento" y "Volatilidad" para cada empresa
for company in companies:
    df = dataframes[company]
    df['Rendimiento'] = (df['Adj Close'] / df['Adj Close'].shift(1)) - 1
    df['Volatilidad'] = df['Rendimiento'].rolling(252).std() * (252 ** 0.5)

    # Calcular el índice Sharpe para cada empresa
    df['Indice Sharpe'] = (df['Rendimiento'].mean() / df['Rendimiento'].std()) * (252 ** 0.5)

    # Crear un CSV para cada empresa
    df.to_csv(f"{company}.csv", index=False)

# Combinar los dataframes en un solo dataframe
df_combined = pd.concat(dataframes.values())

# Crear las columnas "Rendimiento" y "Volatilidad" para el dataframe combinado
df_combined['Rendimiento'] = (df_combined.groupby('Ticker')['Adj Close'].pct_change())
df_combined['Volatilidad'] = (df_combined.groupby('Ticker')['Rendimiento'].rolling(252).std() * (252 ** 0.5)).reset_index(level=0, drop=True)

# Calcular el índice Sharpe para el dataframe combinado
df_combined['Indice Sharpe'] = (df_combined.groupby('Ticker')['Rendimiento'].mean() / df_combined.groupby('Ticker')['Rendimiento'].std()) * (252 ** 0.5)

# Exportar el dataframe combinado a CSV
df_combined.to_csv("combined.csv", index=False)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [11]:
import streamlit as st
import pandas as pd

# Leer los archivos CSV generados previamente
df_AOS = pd.read_csv('AOS.csv')
df_AFL = pd.read_csv('AFL.csv')
df_ABT = pd.read_csv('ABT.csv')
df_PTC = pd.read_csv('PTC.csv')

# Crear una función para mostrar los KPI's
def kpis(df):
    st.write(f"Rendimiento: {round(df['Rendimiento'].iloc[-1], 2)}")
    st.write(f"Volatilidad: {round(df['Volatilidad'].iloc[-1], 2)}")
    st.write(f"Índice Sharpe: {round(df['Rendimiento'].iloc[-1] / df['Volatilidad'].iloc[-1], 2)}")

# Crear una página para cada empresa
st.set_page_config(page_title='Dashboard', page_icon=":bar_chart:", layout="wide")

# Página para AOS
st.write("# A.O. Smith Corporation (AOS)")
kpis(df_AOS)

# Página para AFL
st.write("# Aflac Incorporated (AFL)")
kpis(df_AFL)

# Página para ABT
st.write("# Abbott Laboratories (ABT)")
kpis(df_ABT)

# Página para PTC
st.write("# PTC Inc. (PTC)")
kpis(df_PTC)

2023-03-28 20:46:26.837 
  command:

    streamlit run c:\Users\jorge\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
